In [1]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-253ad06d-fd98-9572-e9bb-c87873db54d4)
GPU 1: Tesla T4 (UUID: GPU-ffd7f6b9-80cf-5b0d-8951-ae3bd6ff671f)


In [2]:
%%time

# both
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
# !pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off

# th
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
# !pip install -qqq chromadb==0.4.5 --progress-bar off
# !pip install -qqq pypdf==3.15.0 --progress-bar off
# !pip install -qqq pdf2image==1.16.3 --progress-bar off


# my
! pip install -qq -U faiss-cpu
# ! pip install -qq -U accelerate bitsandbytes einops tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
CPU times: user 3.58 s, sys: 831 ms, total: 4.42 s
Wall time: 4min 30s


In [3]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [4]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [5]:
# !sudo apt-get install poppler-utils

In [6]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
# from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader


# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import AutoTokenizer, TextStreamer, pipeline


print(langchain.__version__)

0.0.266


In [7]:
from langchain.vectorstores import Chroma
from auto_gptq import AutoGPTQForCausalLM
# import torch
# from langchain import HuggingFacePipeline, PromptTemplate
# from langchain.chains import RetrievalQA
# from langchain.document_loaders import PyPDFDirectoryLoader
# from langchain.embeddings import HuggingFaceInstructEmbeddings
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from pdf2image import convert_from_path
# from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:0


In [8]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

In [9]:
# max_len = 8192

In [10]:
# class CFG:
#     # LLMs
#     model_name = 'llama2-13b' # wizardlm, bloom, falcon, llama2-7b, llama2-13b
#     temperature = 0,
#     top_p = 0.95,
#     repetition_penalty = 1.15    

#     # splitting
#     split_chunk_size = 800
#     split_overlap = 0
    
#     # embeddings
#     embeddings_model_repo = 'hkunlp/instructor-base'    

#     # similar passages
#     k = 3

In [11]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:0


In [12]:
# model_repo = 'daryl149/llama-2-13b-chat-hf'
        
# tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

# model = AutoModelForCausalLM.from_pretrained(
#     model_repo,
#     load_in_4bit=True,
#     device_map='auto',
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
#     trust_remote_code=True
# )
        
# max_len = 8192

In [13]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [14]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs={"temperature": 0})

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

In [15]:
### testing model, not using the book yet
### answer is not necessarily related to the book
query = "Give me 5 examples of cool potions and explain what they do"
llm(query)

.

Sure thing! Here are five examples of cool potions that you might find in a fantasy world, along with their effects:

1. Potion of Healing: This potion restores health to the drinker, healing wounds and injuries. It might also have a soothing effect on the mind, reducing stress and fatigue.
2. Potion of Strength: This potion grants the drinker temporary superhuman strength, allowing them to lift heavy objects or perform feats of physical prowess. It might also increase endurance, allowing the drinker to work or fight for longer periods without tiring.
3. Potion of Speed: This potion increases the drinker's speed and agility, allowing them to move quickly and dodge attacks more easily. It might also grant the ability to run faster, jump higher, or move through difficult terrain with ease.
4. Potion of Invisibility: This potion makes the drinker temporarily invisible, allowing them to sneak past enemies undetected or escape dangerous situations unnoticed. It might also grant the abili

".\n\nSure thing! Here are five examples of cool potions that you might find in a fantasy world, along with their effects:\n\n1. Potion of Healing: This potion restores health to the drinker, healing wounds and injuries. It might also have a soothing effect on the mind, reducing stress and fatigue.\n2. Potion of Strength: This potion grants the drinker temporary superhuman strength, allowing them to lift heavy objects or perform feats of physical prowess. It might also increase endurance, allowing the drinker to work or fight for longer periods without tiring.\n3. Potion of Speed: This potion increases the drinker's speed and agility, allowing them to move quickly and dodge attacks more easily. It might also grant the ability to run faster, jump higher, or move through difficult terrain with ease.\n4. Potion of Invisibility: This potion makes the drinker temporarily invisible, allowing them to sneak past enemies undetected or escape dangerous situations unnoticed. It might also grant t

In [16]:
prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["context", "question"]
)

In [17]:
def loadForEmbeddings(book_txt):
    # load document
    loader = TextLoader(book_txt, encoding="utf-8")
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap = 0
    )

    texts = text_splitter.split_documents(documents)
    return texts


In [18]:
def wrap_text_preserve_newlines(text, width=200): # 110
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    
    sources_used = llm_response['source_documents'][0].metadata['source']
    
    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [19]:
def llm_ans(query):
    start = time.time()
    qa_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
        retriever = retriever, 
        chain_type_kwargs = {"prompt": PROMPT},
        return_source_documents = True,
        verbose = False
    )
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

In [20]:
import requests
from bs4 import BeautifulSoup
import difflib

# Function to search for a book by name and return the best match URL
def search_book_by_name(book_name):
    base_url = "https://www.gutenberg.org/"
    search_url = base_url + "ebooks/search/?query=" + book_name.replace(" ", "+") + "&submit_search=Go%21"
    
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the best match link based on similarity ratio
    best_match_ratio = 0
    best_match_url = ""

    for link in soup.find_all("li", class_="booklink"):
        link_title = link.find("span", class_="title").get_text()
        similarity_ratio = difflib.SequenceMatcher(None, book_name.lower(), link_title.lower()).ratio()
        if similarity_ratio > best_match_ratio:
            best_match_ratio = similarity_ratio
            best_match_url = base_url + link.find("a").get("href")

    return best_match_url

# Function to get the "Plain Text UTF-8" download link from the book page
def get_plain_text_link(book_url):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    plain_text_link = ""
    
    for row in soup.find_all("tr"):
        format_cell = row.find("td", class_="unpadded icon_save")
        if format_cell and "Plain Text UTF-8" in format_cell.get_text():
            plain_text_link = format_cell.find("a").get("href")
            break
    
    return plain_text_link


# Function to get the content of the "Plain Text UTF-8" link
def get_plain_text_content(plain_text_link):
    response = requests.get(plain_text_link)
    content = response.text
    return content


# Main function
def load_book(book_name):
    best_match_url = search_book_by_name(book_name)

    if best_match_url:
        plain_text_link = get_plain_text_link(best_match_url)
        if plain_text_link:
            full_plain_text_link = "https://www.gutenberg.org" + plain_text_link
            plain_text_content = get_plain_text_content(full_plain_text_link)
#             print("Plain Text UTF-8 content:", plain_text_content)
            
            book_text = plain_text_content
            
            file = book_name + ".txt"

            # Remove the BOM character if it exists
            book_text = book_text.lstrip('\ufeff')

            # Choose an appropriate encoding, such as 'utf-8'
            with open(file, "w", encoding="utf-8") as file:
                file.write(book_text)
                
            return book_text
        else:
            print("No Plain Text UTF-8 link found.")
            return "web site error"
    else:
        print("No matching book found.")
        return "web site error"

# def clean_book_content(book_text):
#     cleaned_book_text = book_text.replace("\n", " ")
#     cleaned_book_text = cleaned_book_text.replace("\r", " ")
#     cleaned_book_text = cleaned_book_text.replace("\ufeff", "")
#     return cleaned_book_text

In [21]:
book_name = "The prince"
book_text = load_book(book_name)
file = book_name + ".txt"
texts = loadForEmbeddings(file)

### download embeddings model
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name = 'hkunlp/instructor-base',
    model_kwargs = {"device": "cuda"}
)

### create embeddings and DB
vectordb = FAISS.from_documents(
    documents = texts, 
    embedding = instructor_embeddings
)

### persist vector database
vectordb.save_local("faiss_index_hp")
    
retriever = vectordb.as_retriever(search_kwargs = {"k": 3, "search_type" : "similarity"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [22]:
llm_ans("What are the characters")

The characters mentioned are princes, the people, and mercenary captains.


' The characters mentioned are princes, the people, and mercenary captains.\n\nSources: \nThe prince.txt\n\nTime elapsed: 2 s'

In [23]:
# pip install gradio

In [24]:
# import gradio as gr

# title = "GutenbergChat Hub"
# vectordb = None
# retriever = None

# # Submit book
# def submit_book(book_name):
#     global vectordb, retriever
#     if not book_name:
#         return "Please enter the name of the book."
    
#     book_text = load_book(book_name)
#     file = book_name + ".txt"
#     texts = loadForEmbeddings(file)

#     ### download embeddings model
#     instructor_embeddings = HuggingFaceInstructEmbeddings(
#         model_name = 'hkunlp/instructor-base',
#         model_kwargs = {"device": "cuda"}
#     )

#     ### create embeddings and DB
#     vectordb = FAISS.from_documents(
#         documents = texts, 
#         embedding = instructor_embeddings
#     )

#     ### persist vector database
#     vectordb.save_local("faiss_index_hp")
    
#     retriever = vectordb.as_retriever(search_kwargs = {"k": 3, "search_type" : "similarity"})

    
#     return "done"


# # Q/A
# def get_response(prompt):
# #     if (not book_name and not prompt):
# #         return "Please enter the name of the book and the prompt."
# #     if not book_name:
# #         return "Please enter the name of the book."
# #     if not prompt:
# #         return "Please enter the prompt."
#     query = prompt
# #     llm_response = qa_chain(query)
    
#     return llm_ans(query)

# # Interface 1
# submitBook = gr.Interface(fn=submit_book, inputs="text", outputs="text", title="Submit your book here first")
# # Interface 2
# chatBot = gr.Interface(
#         fn=get_response,
#         inputs="text",
#         outputs="text",
#         title=title + " - Q/A-Bot",
#         description="Enter the name of the book in previous tab, then ask questions here",
#         examples=["What are the characters in the book?"]
#     )

# demo = gr.TabbedInterface([submitBook, chatBot], ["SubmitBook", "Q/A-Bot"])
# demo.launch()